In [1]:
import socket
import threading
import tkinter
import tkinter.scrolledtext
from tkinter import simpledialog

In [3]:
HOST='192.168.1.41'
PORT= 1111

class Client:
    def __init__(self, host, port):
        self.sock=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((host,port))
        message=tkinter.Tk()
        message.withdraw()

        self.nickname=simpledialog.askstring("nickname", "choose a name", parent=message)#What is parent code?
        
        self.gui_done= False
        
        self.running=True
        
        gui_thread= threading.Thread(target=self.gui_loop)
        receive_thread= threading.Thread(target=self.receive)
        
        gui_thread.start()
        receive_thread.start()
        
    def gui_loop(self):
        self.win=tkinter.Tk()
        self.win.configure(bg="gray")
        
        self.chat_label= tkinter.Label(self.win, text="Chat:",bg="gray")
        self.chat_label.pack(padx=20, pady=5)

        self.text_area= tkinter.scrolledtext.ScrolledText(self.win)
        self.text_area.pack(padx=20, pady=5)

        self.message_label= tkinter.Label(self.win, text="Message:", bg="gray")
        self.message_label.pack(padx=20, pady=5)

        self.input_area=tkinter.Text(self.win, height=5)
        self.input_area.pack(padx=20, pady=5)
        
        self.send_button=tkinter.Button(self.win, text="Okay", command= self.write)
        self.send_button.pack(padx=20, pady=5)
        
        self.gui_done=True
        
        self.win.protocol("WM_DELETE_WINDOW", self.stop)
        
        self.win.mainloop()

    

    def write(self):
        message= f"{self.nickname}: {self.input_area.get('1.0', 'end')}"
        self.sock.send(message.encode('utf-8'))
        self.input_area.delete('1.0', 'end')
        
    def stop(self):
        self.running= False
        self.win.destroy()
        self.sock.close()
        exit(0)
        
    
    def receive(self):
        while self.running:
            try:
                message=self.sock.recv(1024).decode('utf-8')
                if message== 'NAME YOU WANT':
                    self.sock.send(self.nickname.encode('utf-8'))
                else: 
                    if self.gui_done:
                        self.text_area.config(state='normal')
                        self.text_area.insert('end', message)
                        self.text_area.yview('end')
                        self.text_area.config(state='disabled')
            except ConnectionAbortedError:
                break
            except:
                print("ERROR")
                self.sock.close()
                break

client=Client(HOST, PORT)